In [48]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scipy as sp
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import proplot as pplt
from matplotlib.font_manager import FontProperties as FP
import warnings
import math

In [2]:
# font
fs = 10
FS_label = fs+1
Axis_FS =fs-1
LEG_FS = fs-2
Annotation_FS = fs-0.5

# tick line width
width_major = 0.4
width_minor = 0.2
LW = np.arange(0, 3.0, 0.25)

# data control
InsCase = 0 # 0 -> Case A(25 cm) # 1 -> Case B(30 cm)
path1 = "output_summer"

# unit change 
day_to_hour = 24
hour_to_min = 60
hour_to_sec = 3600
min_to_sec = 60  
sec_to_hour = 1/3600        

# set timestep, duration
t_step = 10 #[s] 
PSTime = 120 #[h] PS: PreSimulation 
TSTime = 144 #[h] TS: TotalSimulation
simulation_time = TSTime - PSTime # [h]
PSTimeRowNum = int(PSTime*hour_to_sec/t_step)
TSTimeRowNum = int((TSTime)*hour_to_sec/t_step)
t_row_end = int((TSTime)*hour_to_sec/t_step) # 시간 행
t_list = [t_step*i for i in range(t_row_end+1)] #index 0 부터 시작 -> t_row_end+1
time = pd.DataFrame(t_list) #second
time_min = time/60; #minute
time_hour = time/3600; #hour
time_end = time_hour[-1:];    # last time

def cm2in(value):
    return value/2.54
def list_sec_to_hour(list):
    return([list[i]*sec_to_hour for i in range(len(list))])

### Excel import

In [3]:
U_q_a= pd.read_excel("alpha=0.5/DF_U_q.xlsx",  sheet_name="20 cm")
U_q_b = pd.read_excel("alpha=2.0/DF_U_q.xlsx", sheet_name="20 cm")
U_q_c = pd.read_excel("alpha=3.5/DF_U_q.xlsx", sheet_name="20 cm")
U_q_d = pd.read_excel("alpha=5.0/DF_U_q.xlsx", sheet_name="20 cm")

U_T_a= pd.read_excel("alpha=0.5/DF_U_T.xlsx",  sheet_name="20 cm")
U_T_b = pd.read_excel("alpha=2.0/DF_U_T.xlsx", sheet_name="20 cm")
U_T_c = pd.read_excel("alpha=3.5/DF_U_T.xlsx", sheet_name="20 cm")
U_T_d = pd.read_excel("alpha=5.0/DF_U_T.xlsx", sheet_name="20 cm")

In [4]:
D_q_a = pd.read_excel("alpha=0.5/DF_D_q.xlsx",  sheet_name="20 cm")
D_q_b = pd.read_excel("alpha=2.0/DF_D_q.xlsx", sheet_name="20 cm")
D_q_c = pd.read_excel("alpha=3.5/DF_D_q.xlsx", sheet_name="20 cm")
D_q_d = pd.read_excel("alpha=5.0/DF_D_q.xlsx", sheet_name="20 cm")

D_T_a = pd.read_excel("alpha=0.5/DF_D_T.xlsx",  sheet_name="20 cm")
D_T_b = pd.read_excel("alpha=2.0/DF_D_T.xlsx", sheet_name="20 cm")
D_T_c = pd.read_excel("alpha=3.5/DF_D_T.xlsx", sheet_name="20 cm")
D_T_d = pd.read_excel("alpha=5.0/DF_D_T.xlsx", sheet_name="20 cm")

In [106]:
# [단열재종류][In,Mid,Out]
# Temp = [U_T_a, D_T_a]
# Heatflux = [U_q_a, D_q_a]
Temp = [U_T_b, D_T_b]
Heatflux = [U_q_b, D_q_b]

In [107]:
# meshgrid 써서 x_pos y_pos 좌표계 작성 
thin_int  = 360    # thin out interval
PSTimeRowNum = int(PSTime*hour_to_sec/t_step)
TSTimeRowNum = int((TSTime)*hour_to_sec/t_step)

system_length = Heatflux[0].shape[1] #Count 

time_hour_int = time_hour[PSTimeRowNum:TSTimeRowNum+thin_int:thin_int].values
tint = time_hour_int[1][0]-time_hour_int[0][0]

x = np.arange(0.5,system_length,1)   # x coordinate nodes
y = time_hour_int
x_pos1, y_pos1 = np.meshgrid(x, y)
x_pos2, y_pos2 = np.meshgrid(x, y)  
x_pos3, y_pos3 = np.meshgrid(x, y) 
x_pos4, y_pos4 = np.meshgrid(x, y) 

x_pos = [x_pos1,x_pos2,x_pos3,x_pos4]
y_pos = [y_pos1,y_pos2,y_pos3,y_pos4]

# normalize constant
# 왼쪽엑서지 플로우는 Heatflux 부호가 -, 오른쪽 플로는 Heatflux부호가 +, 플럭스 크기에 따라서 적절히 노말라이즈 해줘야함. 
norm_const1 = Heatflux[0][::thin_int].abs().max().max()    # normalize용 최대값 
norm_const2 = Heatflux[1][::thin_int].abs().max().max()    # normalize용 최대값
# norm_ratio = norm_const1/norm_const3    # 비교하는 케이스 사이의 max 비율
norm_const1_ceil = int(np.ceil(norm_const1*100))/100   #cmap 레젠드 위해서 올림
norm_const2_ceil = int(np.ceil(norm_const2*100))/100   #cmap 레젠드 위해서 올림
norm_max = max([norm_const1_ceil,norm_const2_ceil,])
norm_const_ceil = [norm_const1_ceil,norm_const2_ceil]

# u direction 
# x축 방향 0 
u1 = Heatflux[0][::thin_int]
u2 = Heatflux[1][::thin_int]

u = [u1,u2]

# v direction 
# st 는 y 방향으로 벡터 가짐 
v1 = np.zeros(y_pos1.shape) # slicing을 통해 벡터 일정 간격으로 생성
v2 = np.zeros(y_pos2.shape) # slicing을 통해 벡터 일정 간격으로 생성
v3 = np.zeros(y_pos3.shape) # slicing을 통해 벡터 일정 간격으로 생성
v4 = np.zeros(y_pos4.shape) # slicing을 통해 벡터 일정 간격으로 생성

v = [v1,v2,v3,v4]

# temperature field 
temp1_hourly = Temp[0][::thin_int] # slicing
temp2_hourly = Temp[1][::thin_int] # slicing

temp_hourly = [temp1_hourly,temp2_hourly,]

# normalize constant
# 왼쪽엑서지 플로우는 Xf 부호가 -, 오른쪽 플로는 Xf부호가 +, 플럭스 크기에 따라서 적절히 노말라이즈 해줘야함. 
norm_const1 = Heatflux[0][::thin_int].abs().max().max()    # normalize용 최대값 
norm_const2 = Heatflux[1][::thin_int].abs().max().max()    # normalize용 최대값
norm_const1_ceil = round(norm_const1)   #cmap 레젠드 위해서 올림
norm_const2_ceil = round(norm_const2)   #cmap 레젠드 위해서 올림

# temperature field 
temp1_hourly = Temp[0][::thin_int] # slicing
temp2_hourly = Temp[1][::thin_int] # slicing
temp_hourly = [temp1_hourly,temp2_hourly]
# hourly environmental temp
# temp_env1_hourly = temp_env1[::thin_int]
# temp_env3_hourly = temp_env3[::thin_int]

# quiver color array
# 카르노효율의 부호가 warm/cool인지 결정하니까 색깔 c_array의 부호로 관련정보 담아야함
# Xf 전부 양수로 절대값 씌우고 sign_Carneff_flux1 곱함.
# c_array1 = Heatflux[0].abs()*sign_Carneff_flux1
# c_array1 = c_array1[::thin_int]
# c_array2 = Heatflux[1].abs()*sign_Carneff_flux2
# c_array2 = c_array2[::thin_int]
# c_array3 = Heatflux[2].abs()*sign_Carneff_flux3
# c_array3 = c_array3[::thin_int]

In [108]:
print(norm_const1_ceil)
print(norm_const2_ceil)

57
50


In [109]:
import matplotlib.colors as mcolors

In [113]:
norm1 = mcolors.Normalize(vmin = 0, vmax = 60)
norm2 = mcolors.Normalize(vmin = 10, vmax = 30)
temcol= sns.light_palette('#333333', as_cmap=True)

In [114]:
nrows = 1
ncols = 2
nfigs = nrows*ncols
# subplot  
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(cm2in(18),cm2in(14)), 
                        facecolor='w', edgecolor='k', squeeze=False, dpi =600)

norm_const1 = Heatflux[0][::thin_int].abs().max().max()    # normalize용 최대값 
norm_const2 = Heatflux[1][::thin_int].abs().max().max()    # normalize용 최대값 
norm_const = [norm_const1,norm_const2]
# quiver params
wd = 0.15 # unit 알고리즘이랑 width랑 연동됨: 화살표 촉 (shaft) 너비를 커스텀 
sc1 = norm_const1*1.1 # scale은 화살 길이에 대한 척도로 작을 수록 길어짐: 서브플랏으로 두케이스 비교할 경우에는 통일해야함
sc2 = norm_const2*1.1 # scale은 화살 길이에 대한 척도로 작을 수록 길어짐: 서브플랏으로 두케이스 비교할 경우에는 통일해야함
sc = [sc1,sc2,]
norm_const_ceil = [norm_const1_ceil,norm_const2_ceil,]
head_wd = 2.2 # 화살표 너비
interp = 'none'

vx_pos = 0.40
for ridx in range(nrows):
     for cidx in range(ncols):
          idx = ncols*ridx + cidx
          q = ax[ridx,cidx].quiver(x_pos[idx], y_pos[idx] ,u[idx], v[idx], abs(u[idx]), cmap='YlOrRd', #Fire, YIOrRd,Reds1,Reds2 
                                   norm=norm1, units ='xy', scale=sc[idx], width=wd, headwidth=head_wd) 
          ax[ridx,cidx].invert_yaxis()    # reverse y axis
          ax[ridx,cidx].quiverkey(q, X=vx_pos, Y=1.023, U=-norm_const_ceil[idx], 
            label=f': {norm_const_ceil[idx]} W/m$^2$', labelpos='E', labelsep=0.2,
            fontproperties = FP(size=fs-1))  
          ax[ridx,cidx].quiverkey(q, X=vx_pos+0.35, Y=1.023, U=-norm_const_ceil[idx]/2, 
            label=f': {int(norm_const_ceil[idx]/2)} W/m$^2$', labelpos='E', labelsep=0.14,
            fontproperties = FP(size=fs-1))        
         # temperature field
          tf = ax[ridx,cidx].imshow(temp_hourly[idx], 
                              aspect='auto', extent=[0,system_length,TSTime+tint/2,PSTime-tint/2], 
            cmap=temcol, norm=norm2, alpha=1.0, interpolation=interp)
          

# Axes labels 
for ridx in range(nrows):
    for cidx in range(ncols):        
        # axis label
        ax[0,cidx].set_xlabel('x coordinate [cm]', fontsize = fs)
        ax[ridx,0].set_ylabel('Elapsed time [h]', fontsize = fs)

# tick variables
xtick_decimal = ['0','0','3','0','0']
ytick_decimal = ['0','0','0','1','1']
xmin = [-0.25]*nfigs
xmax = [system_length+0.25]*nfigs
xint = [5]*nfigs
xmar = [0.5]*nfigs

ymin = [PSTime]*nfigs
ymax = [TSTime]*nfigs 
yint = [6]*nfigs
ymar = [tint*1.2]*nfigs

# figure settings 
for ridx in range(nrows):
    for cidx in range(ncols):
        ax[ridx,cidx].grid(False)
        idx = ridx*ncols + cidx
        ax[ridx,cidx].tick_params(direction='in', labelsize=Axis_FS, which='major', length=3, width=0.5)
        ax[ridx,cidx].tick_params(direction='in', labelsize=Axis_FS, which='minor', length=1.5,width=0.25)
        
        # xlim ylim 
        ax[ridx,cidx].set_xlim(xmin[idx]-xmar[idx], xmax[idx]+xmar[idx])
        ax[ridx,cidx].set_ylim(ymax[idx]+ymar[idx], ymin[idx]-ymar[idx])     
        
        # tick setting
        # number of major ticks
        # ax[ridx,cidx].xaxis.set_major_locator(ticker.MaxNLocator(6))
        # ax[ridx,cidx].yaxis.set_major_locator(ticker.MaxNLocator(5))
        
        # ax[ridx,cidx].set_xticks(np.arange(xmin[idx], xmax[idx]+xint[idx], xint[idx])) 
        ax[ridx,cidx].set_xticks(np.arange(0, system_length+5, 5).tolist()) 
        ax[ridx,cidx].set_yticks(np.arange(ymin[idx], ymax[idx]+yint[idx], yint[idx])) 
        # tick decimal
        ax[ridx,cidx].xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.'+xtick_decimal[idx]+'f'))
        ax[ridx,cidx].yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.'+ytick_decimal[idx]+'f'))
        
        # replace xtick string 
        xtick_list = ax[ridx,cidx].get_xticks().tolist()
        ytick_list = ax[ridx,cidx].get_yticks().tolist()
        ax[ridx,0].set_xticklabels(xtick_list)
        ax[ridx,cidx].set_yticklabels(ytick_list)
        
        # number of minor ticks
        ax[ridx,cidx].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
        ax[ridx,cidx].yaxis.set_minor_locator(ticker.AutoMinorLocator(3))

        # grid 
#         grid line clipping https://matplotlib.org/users/dflt_style_changes.html
            
        # subplot annotation, https://matplotlib.org/gallery/text_labels_and_annotations/annotation_demo.html 
#        subplot_idx = '('+subplot_no[2*ridx+cidx]+')'
        # annotation
        subplot_no = list(map(chr,range(97,123)))
        case_name = ['US', 'D']
        subplot_idx = '('+subplot_no[idx]+') ' + case_name[idx] #(a) + Internal
        ax[ridx,cidx].annotate(subplot_idx, xy=(.01, 1.037), xycoords='axes fraction',
            horizontalalignment='left', verticalalignment='top', fontsize=Annotation_FS) 
                 
        # spine line width  
        for k in ['top','bottom','left','right']:
                ax[ridx,cidx].spines[k].set_visible(True)
                ax[ridx,cidx].spines[k].set_linewidth(0.5)
                ax[ridx,cidx].spines[k].set_color('k')
                
        # title_matrix = ['unsteady state', 'steady state'] + [""]*(nfigs-ncols)
        # title = ax[ridx,cidx].set_title(title_matrix[idx], fontdict={'fontsize': fs}, pad=20)
        # title.set_position((0.5,-1))

# plt.subplots_adjust(hspace=0.15, wspace = 0.08)
plt.subplots_adjust(wspace=0.1,hspace=0.18, right=0.9, left=0.1,bottom=0.1, top=0.95)
fig.subplots_adjust(right=0.85, bottom=0.21)

# colorbar
cbar_width =  0.015 #vertical  
cbar_height = 0.020 #horizontal  
cbar_dist_v = 0.13; # vertical colorbar distance from bbox edge
cbar_dist_h = 0.03; # horizontal colorbar distance from bbox edge

# horizontal colorbars 
# colorbar1 temperature 
bbox1 = ax[0,0].get_position() # get the normalized axis original position 
bbox3 = ax[0,1].get_position() # get the normalized axis original position 
cb_ax1 = fig.add_axes([bbox1.x0, bbox1.y0-cbar_dist_v, bbox3.x1-bbox1.x0, cbar_height]) #[x_origin, y_origin, width, height]
cbar1  = fig.colorbar(tf, cax=cb_ax1, ax=ax[0,0], orientation='horizontal')
cbar1.ax.tick_params(which = "major" ,direction='in',labelsize=Axis_FS, length = 3, width = 0.5)
cbar1.ax.minorticks_off()
cbar1.ax.set_xlabel('Node temperature [$^{\circ}$C]', rotation=0, fontsize=FS_label, labelpad=5)
cbar1.locator = ticker.MultipleLocator(2)
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

# vertical colorbars 
# colorbar2 heatflux  
bbox1 = ax[0,1].get_position() # get the normalized axis original position 
bbox3 = ax[0,1].get_position() # get the normalized axis original position 
cb_ax2 = fig.add_axes([bbox3.x1+cbar_dist_h, bbox3.y0, cbar_width, bbox1.y1-bbox3.y0]) #[x_origin, y_origin, width, height]
cbar2  = fig.colorbar(q, cax=cb_ax2, ax=ax[0,0], orientation='vertical') 
cbar2.ax.tick_params(which = "major" ,direction='in',labelsize=Axis_FS, length = 3, width = 0.5)
cbar2.ax.minorticks_off()
cbar2.locator = ticker.MultipleLocator(10)
cbar2.ax.set_ylabel('Heat flux [W/m$^2$]', rotation=90, fontsize=FS_label, labelpad=1)
cbar2.update_ticks()
cbar2.outline.set_linewidth(0.5)